In [14]:
from pydantic import BaseModel, Field, ValidationError

from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [15]:
class DocumentInput(BaseModel):
    question: str = None


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

tools = []
files = [
    # https://abc.xyz/investor/static/pdf/2023Q1_alphabet_earnings_release.pdf
    {
        "name": "Resume Vega",
        "path": "PDFS/Resume_MongoDB.pdf",
    },
    # https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q1-2023-Update
    {
        "name": "Resume Omar",
        "path": "PDFS/Omar_Macias_Resume.pdf",
    },
]



In [16]:
for file in files:
    loader = PyPDFLoader(file["path"])
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(pages)
    embeddings = OpenAIEmbeddings()
    retriever = FAISS.from_documents(docs, embeddings).as_retriever()

    # Wrap retrievers in a Tool
    tools.append(
        Tool(
            #args_schema=DocumentInput(),
            name=file["name"],
            description=f"useful when you want to answer questions about {file['name']}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
        )
    )

In [17]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)

agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
)



In [18]:
query = "Como se llaman?"

In [19]:
agent.run(query)



> Entering new AgentExecutor chain...


InvalidRequestError: 'Resume Vega' does not match '^[a-zA-Z0-9_-]{1,64}$' - 'functions.0.name'